# Up Next

- make a day of week chart 
- Get better colors/labels/titles for visuals
- How up-to-date are RRCU's txns? will I need to wait a few days into month?
    - looking like at least two days for credit card txns
- Make the categorization code into a class
- Maybe automatic email

Categories <br>
shopping, tithing, subscriptions, progressive, fun, power, gas, water, unknown, wmt, internet, amazon, rent, food, car 

In [1]:
import polars as pl
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, date2num
import time
import csv
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

##### Check this once, then dump it

In [2]:

# End date
days_backward = time.localtime().tm_mday
now = datetime.now()
EndOfLastMonth = now - timedelta(days=days_backward)
# Start date
LastYear = EndOfLastMonth - relativedelta(years=1)
StartDate = LastYear + relativedelta(months=1)
StartDate = StartDate.replace(day=1).date()

print('startdate: ', StartDate)
print('endDate: ', EndOfLastMonth)

startdate:  2024-05-01
endDate:  2025-04-30 19:07:10.017652


In [199]:
df = pl.read_csv(r'C:\Users\Blake Dennett\Downloads\BankStreamline\Data\BankDataProd.csv')

df = df.with_columns(
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d", strict=False)
)


# End date
days_backward = time.localtime().tm_mday
now = datetime.now()
EndOfLastMonth = now - timedelta(days=days_backward)
# Start date
LastYear = EndOfLastMonth - relativedelta(years=1)
StartDate = LastYear + relativedelta(months=1)
StartDate = StartDate.replace(day=1).date()


df = df.filter(pl.col("date") >= pl.date(StartDate.year, StartDate.month, StartDate.day)) \
        .filter(pl.col("date") <= pl.date(EndOfLastMonth.year, EndOfLastMonth.month, EndOfLastMonth.day))

df.limit(3)

date,category,description,cost,controllable,monthName,month,day,year,weekDay,cardType
date,str,str,f64,i64,str,i64,i64,i64,str,str
2025-03-31,"""food""","""DC THE BLACK C…",54.61,1,"""Mar""",3,31,2025,"""Mon""","""Debit"""
2025-03-31,"""amazon""","""DC eBay O*08-1…",125.74,1,"""Mar""",3,31,2025,"""Mon""","""Debit"""
2025-03-28,"""wmt""","""POS 0328 0135 …",58.47,1,"""Mar""",3,28,2025,"""Fri""","""Debit"""


#### Check dates

In [201]:
print(list(df['month'].unique()))
print(list(df['monthName'].unique()))
print(list(df['weekDay'].unique()))
print(list(df['year'].unique()))
print(str(df.select(pl.col("date").min())))
print(df.select(pl.col("date").max()))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
['Nov', 'Oct', 'Mar', 'Aug', 'Dec', 'Jul', 'Jan', 'May', 'Jun', 'Sep', 'Apr', 'Feb']
['Wed', 'Fri', 'Sun', 'Sat', 'Thu', 'Tue', 'Mon']
[2024, 2025]
shape: (1, 1)
┌────────────┐
│ date       │
│ ---        │
│ date       │
╞════════════╡
│ 2024-04-01 │
└────────────┘
shape: (1, 1)
┌────────────┐
│ date       │
│ ---        │
│ date       │
╞════════════╡
│ 2025-03-31 │
└────────────┘


# Month Last Pie Chart

In [202]:
fig = px.pie(df, values='cost', names='category', template='plotly_dark', width=600, height=400)
fig.show()

In [203]:
df.filter(pl.col('category') == 'gas').select(pl.col('cost').sum())
2270.89/11.5

197.4686956521739

## Expense categories by month

In [204]:
import plotly.express as px

# Define the correct month order
month_order = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Sort the dataframe by month number to ensure correct order
df = df.sort("month")

df = df.filter(pl.col('category') == 'gas')
# df = df.filter(pl.col('controllable') > 0)

# Create the plot with an explicit category order
fig = px.bar(df.to_pandas(), x="monthName", y="cost", color="category", hover_data=['description', 'cardType', 'day'],
             title="Total Expense by Month",
             category_orders={"monthName": month_order},
             template='plotly_dark',
             text_auto=True)  # This adds text labels on top of the bars

fig.show()

In [205]:
# # way to make totals at the top
# # Aggregate total cost per month
# monthly_totals = df.to_pandas().groupby("monthName")["cost"].sum().reset_index()

# # Create the bar chart
# fig = px.bar(df.to_pandas(), x="monthName", y="cost", color="category", hover_data=['description', 'cardType', 'day'],
#              title="Total Expense by Month",
#              category_orders={"monthName": month_order})

# # Add text labels for monthly totals
# fig.add_trace(go.Scatter(
#     x=monthly_totals["monthName"], 
#     y=monthly_totals["cost"],
#     text=monthly_totals["cost"],
#     mode='text',  
#     textposition="top center", 
#     showlegend=False  # Hide legend for this trace
# ))

# fig.show()

#### Expense by month

In [206]:
import plotly.express as px

# Define the correct month order
month_order = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Sort the dataframe by month number to ensure correct order

# df = df.filter(pl.col('controllable') > 0)

# gr = df.filter(pl.col('category') == 'rent')

gr = df.group_by(["monthName", 'month']).agg(pl.sum("cost")) 

gr = gr.sort("month")


# Create the plot with an explicit category order
fig = px.bar(gr.to_pandas(), x="monthName", y="cost",
             title="Total Expense by Month",
             template='plotly_dark',
             color_discrete_sequence=['#228B22'])
            #  category_orders={"monthName": month_order},
            #  text_auto=True)  # This adds text labels on top of the bars

# # Add text labels for monthly totals
fig.add_trace(go.Scatter(
    x=gr["monthName"], 
    y=gr["cost"],
    text=gr["cost"].to_pandas().round(0).astype(int),
    mode='text',  
    textposition="top center", 
    showlegend=False  # Hide legend for this trace
))

avg_cost = gr["cost"].mean()

fig.add_shape(
    type="line",
    x0=month_order[0], x1=month_order[-1],  # Span the full width
    y0=avg_cost, y1=avg_cost,  # Keep it at the average level
    line=dict(color="white", width=2, dash="dash")  # Customize the line style
)

fig.add_annotation(
    x=month_order[-1], y=avg_cost + (avg_cost * .1),
    text=f"Avg: {avg_cost:.0f}",
    showarrow=False,
    font=dict(size=12, color="white")
)

fig.show()

# Day of week

In [207]:
import plotly.express as px

# Define the correct month order
month_order = ["Mon", 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# Sort the dataframe by month number to ensure correct order

# df = df.filter(pl.col('controllable') > 0)

# gr = df.filter(pl.col('category') == 'gas')

gr = df.group_by(['weekDay']).agg(pl.sum("cost")) 

gr = gr.sort("weekDay")


# Create the plot with an explicit category order
fig = px.bar(gr.to_pandas(), x="weekDay", y="cost",
             title="Total Expense by Month",
             template='plotly_dark',
             color_discrete_sequence=['#228B22'],
             category_orders={"weekDay": month_order},)
            #  text_auto=True)  # This adds text labels on top of the bars

# # Add text labels for monthly totals
fig.add_trace(go.Scatter(
    x=gr["weekDay"], 
    y=gr["cost"],
    text=gr["cost"].to_pandas().round(0).astype(int),
    mode='text',  
    textposition="top center", 
    showlegend=False  # Hide legend for this trace
))

avg_cost = gr["cost"].mean()

fig.add_shape(
    type="line",
    x0=month_order[0], x1=month_order[-1],  # Span the full width
    y0=avg_cost, y1=avg_cost,  # Keep it at the average level
    line=dict(color="white", width=2, dash="dash")  # Customize the line style
)

# Add a text annotation to label the line
fig.add_annotation(
    x=month_order[-1], y=avg_cost + 200,
    text=f"Avg: {avg_cost:.0f}",  # Format to round off decimals
    showarrow=False,
    font=dict(size=12, color="white")
)

fig.show()

#### Unknowns

In [208]:
list(df.filter(pl.col('category') == 'unknown')['description'].unique())

[]

In [209]:
list(df.filter((pl.col('category') == 'unknown') & (pl.col('month') == 2))['description'].unique())


[]

#### Junk